此程式
買:當五日平均線反轉，並且斜率差超過25%則買進
    
賣:每當賺超過20%賣掉，3%止損

In [2]:
import os
import numpy as np # 全買全賣 五日平均反轉點by 斜率
import pandas as pd
import mplfinance as mpf
from datetime import datetime
import math as mt
def build_detail(stocks, cur_data, unit, profit,cash,buy):
    temp_detail = []
    temp_detail.append(str(datetime.date(stocks['Date'][cur_data]).month) + '/' +
                        str(datetime.date(stocks['Date'][cur_data]).day))                     #Date日期
    if(buy == True):
        temp_detail.append('+' + str(unit))                                               #Buy unit stock 買單位
    else:
        temp_detail.append('-' + str(unit))                                               #Sell unit stock賣單位

    temp_detail.append(stocks_close[cur_data])                                               #Buying price價格
    temp_detail.append(round(stocks_close[cur_data] * 1.001425, 1))                          #Tax included price含稅價
    if(buy == True):
        temp_detail.append('-' + str(round(stocks_close[cur_data] * 1.001425 * unit*1000)))    #Actual cost實際花費
    else:
        temp_detail.append('+' + str(round(stocks_close[i] * 0.995575 * unit*1000)))
    temp_detail.append(profit)                                                        #Profit 獲利
    temp_detail.append(cash)
    return temp_detail

path = os.getcwd() # 取得此檔案當前的工作路徑
list_csv = os.listdir(path) # 取得此路徑中的所有檔案
lists = [file for file in list_csv if '.csv' in file] # 過濾出股價檔(.csv)
for file in lists:
    print(file)
    stocks = pd.read_csv(file)
    stocks = stocks.set_index(pd.DatetimeIndex(stocks["Date"]))
    stocks.iloc[:, 0] = pd.to_datetime(stocks.iloc[:,0], format="%Y/%m/%d")
    colorSet = mpf.make_marketcolors(up='r', down='g')
    styleSet = mpf.make_mpf_style(marketcolors=colorSet)
    kwargs = dict(type='candle', figscale=2, style=styleSet)
    stocks_close = [float(line) for line in stocks['Close']]
    stocks_high = [float(line) for line in stocks['High']]
    stocks_low = [float(line) for line in stocks['Low']]
    def making_averages(days):
        days_mav = [np.nan] * (days - 1)
        for i in range((days - 1), len(stocks_close)):
            days_mav.append(sum(stocks_close[i - (days - 1): i + 1]) / (days))
        return days_mav
    def countingslope(x1,x2,y1,y2):
        if (x2-x1==0):
            return 0
        m = (y2-y1)/(x2-x1)
        return m
    FiveMav = making_averages(5)  # 5日線
    points_buy = [np.nan] * len(stocks)  # 買入點
    points_sell = [np.nan] * len(stocks)  # 賣出點
    fivemavslope = [np.nan] *len(stocks)
    for i in range(0,len(stocks)):
        fivemavslope[i] = countingslope(i-1,i,FiveMav[i-1],FiveMav[i])
    stock = 0   # 股票張數
    profit = 0 #獲利
    cash = 0    #剩餘金額
    principle = 500000 #本金
    # 紀錄交易資訊
    detail = [['日期','買/賣(+/-)','價格', '價格(稅)','實際花費','目前獲利','剩餘金額']]
    temp_detail = []# 暫時交易資訊
    for i in range(5,len(stocks)):
        if ( fivemavslope[i-1] <fivemavslope[i]and stock == 0 and fivemavslope[i]>0 and fivemavslope[i-1]<0 and fivemavslope[i]-fivemavslope[i-1]>=0.25):
            if(len(detail) == 1):#first data
                stock = mt.floor(principle/ (stocks_close[i] * 1.001425 * 1000)*1000)/1000  # 購買股數
                points_buy[i] = stocks_low[i] * 0.98
                cash = mt.floor(principle - stock * (stocks_close[i] * 1000)* 1.001425)  # 剩餘金額
                profit = cash-principle
                detail.append(build_detail(stocks, i, stock, profit, cash,True))
                temp_detail = []  # 清除暫存交易資訊
            else:
                stock = mt.floor(detail[len(detail)-1][6]/ (stocks_close[i] * 1.001425 * 1000) * 1000) / 1000  # 購買股數
                points_buy[i] = stocks_low[i] * 0.98
                cash = mt.floor(float(detail[len(detail)-1][6]) - stock * (stocks_close[i]* 1000)* 1.001425)  # 剩餘金額
                profit = cash-principle
                detail.append(build_detail(stocks,i,stock,profit,cash,True))
                temp_detail = []  # 清除暫存交易資訊
        if (stock > 0 and (((stocks_close[i] - detail[len(detail) - 1][3]) / detail[len(detail) - 1][3]) > 0.2 or ((stocks_close[i] - detail[len(detail) - 1][3]) / detail[len(detail) - 1][3]) < -0.03)):
            points_sell[i] = stocks_high[i] * 1.02  # 繪製點所使用的陣列， 賣出日最高價 * 1.02方便觀看
            cash = mt.floor(cash + stocks_close[i] * 0.995575 * stock * 1000)  # 淨收益計
            profit = cash - principle
            detail.append(build_detail(stocks, i, stock, profit, cash, False))
            stock = 0
            temp_detail = []  # 清除暫存交易資訊
    if(stock > 0):
        points_sell[i] = stocks_high[i] * 1.02  # 繪製點所使用的陣列， 賣出日最高價 * 1.02方便觀看
        cash =mt.floor(cash + stocks_close[i] * 0.995575 * stock * 1000)  # 淨收益計算
        profit = cash - principle
        detail.append(build_detail(stocks, i, stock, profit, cash,False))
        stock = 0
        temp_detail = []  # 清除暫存交易資訊
    print("支出：",principle, "元")
    print("收入:",detail[len(detail)-1][6],"元")
    print("淨收益:",detail[len(detail)-1][5],"元")
    if(len(detail)>1):
        print("投資報酬率:",(int(detail[len(detail)-1][5])/principle)*100,"%")
    else:
        print("No data")
    detail = pd.DataFrame(detail)
    display(detail)
    detail = []

0050_2018.csv
支出： 500000 元
收入: 602766 元
淨收益: 102766 元
投資報酬率: 20.5532 %


,0,1,2,3,4,5,6
0,日期,買/賣(+/-),價格,價格(稅),實際花費,目前獲利,剩餘金額
1,2/21,+6.126,81.5,81.6,-499980,-499981,19
2,4/26,-6.126,79.05,79.2,+482117,-17864,482136
3,4/30,+6.018,80.0,80.1,-482126,-499991,9
4,10/11,-6.018,77.4,77.5,+463732,-36259,463741
5,10/18,+5.85,79.15,79.3,-463687,-499947,53
6,10/25,-5.85,75.5,75.6,+439721,-60227,439773
7,10/31,+5.662,77.55,77.7,-439714,-499941,59
8,11/23,-5.662,75.35,75.5,+424744,-75198,424802
9,12/13,+5.559,76.3,76.4,-424756,-499955,45


1304_2011.csv
支出： 500000 元
收入: 460216 元
淨收益: -39784 元
投資報酬率: -7.9568 %


,0,1,2,3,4,5,6
0,日期,買/賣(+/-),價格,價格(稅),實際花費,目前獲利,剩餘金額
1,1/14,+13.907,35.9,36.0,-499973,-499973,27
2,2/21,-13.907,43.85,43.9,+607123,107150,607150
3,3/14,+15.747,38.5,38.6,-607123,-499974,26
4,3/15,-15.747,37.0,37.1,+580061,80086,580086
5,4/21,+14.572,39.75,39.8,-580062,-499977,23
6,4/25,-14.572,38.45,38.5,+557814,57837,557837
7,5/13,+14.814,37.6,37.7,-557800,-499964,36
8,5/19,-14.814,36.35,36.4,+536106,36142,536142
9,5/27,+15.54,34.45,34.5,-536116,-499974,26


1762_2010.csv
支出： 500000 元
收入: 755965 元
淨收益: 255965 元
投資報酬率: 51.193 %


...

,0,1,2,3,4,5,6
0,日期,買/賣(+/-),價格,價格(稅),實際花費,目前獲利,剩餘金額
1,1/4,+10.086,49.5,49.6,-499968,-499969,31
2,5/19,-10.086,60.0,60.1,+602482,102513,602513
3,6/12,+8.13,74.0,74.1,-602477,-499965,35
4,6/15,-8.13,70.0,70.1,+566582,66616,566616
5,6/21,+7.677,73.7,73.8,-566601,-499986,14
6,6/22,-7.677,71.2,71.3,+544184,44197,544197
7,8/23,+8.147,66.7,66.8,-544179,-499983,17
8,12/13,-8.147,64.6,64.7,+523967,23984,523984
9,2/26,+9.343,56.0,56.1,-523954,-499970,30


2886_2019.csv
支出： 500000 元
收入: 剩餘金額 元
淨收益: 目前獲利 元
No data


,0,1,2,3,4,5,6
0,日期,買/賣(+/-),價格,價格(稅),實際花費,目前獲利,剩餘金額


2926_2013.csv
支出： 500000 元
收入: 160794 元
淨收益: -339206 元
投資報酬率: -67.8412 %


,0,1,2,3,4,5,6
0,日期,買/賣(+/-),價格,價格(稅),實際花費,目前獲利,剩餘金額
1,3/25,+2.142,233.0,233.3,-499797,-499798,202
2,3/27,-2.142,223.5,223.8,+476619,-23180,476820
3,4/24,+2.532,188.0,188.3,-476694,-499875,125
4,4/26,-2.532,182.5,182.8,+460045,-39830,460170
...,...,...,...,...,...,...,...
86,9/18,-2.229,76.0,76.1,+168654,-331333,168667
87,9/21,+2.216,76.0,76.1,-168656,-499989,11
88,1/21,-2.216,73.6,73.7,+162376,-337614,162386
89,1/27,+2.153,75.3,75.4,-162352,-499966,34
